# Open Street Map Case Study - Calgary, Alberta, Canada

By: Raza Anees

## Map Area

<ul>
    <li><a href="https://www.openstreetmap.org/relation/3227127#map=10/51.0289/-114.0875">Calgary City Boundary</a></li>
    <li><a href="https://mapzen.com/data/metro-extracts/">Calgary Metro Extract</a></li>
</ul>

## Introduction

I will be analyzing the open street map data for Calgary, Alberta. This is my home town and it is a sprawling urban centre with more than 1.1 million people. I'm interested to see some of the descriptions for the data in the city and how much data has been contributed to date.

## Set up Libraries and open/save paths

In [1]:
import xml.etree.cElementTree as ET #for parsing the XML file
from pprint import pprint #for "pretty" results
from collections import defaultdict #for initializing keys in dictionaries
import re #for using regular expressions
import csv #for parsing and writing CSVs
import codecs
import cerberus # for validating
import schema # for setting the schema

OSM_FILE = "calgary_canada.osm"
SAMPLE_OSM = "calgary_sample.osm"

NODES_PATH = "nodes.csv"
NODES_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

## Select a small sample of the file for Auditing

In [71]:
k = 50 #parameter: take every k-th level

def get_element(osm_file, tags=("node", "way", "relation")):
    context = iter(ET.iterparse(osm_file, events = ('start', 'end')))
    _,root = next(context)
    
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [72]:
#obtain every k'th level and write to a file for later analysis
with open(SAMPLE_OSM, 'w') as samp:
    samp.write('<?xml version="1.0" encoding="utf-8"?>\n')
    samp.write("<osm>\n ")
    
    for i, elem in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            samp.write(ET.tostring(elem, encoding='utf-8'))
    samp.write('</osm>')

Get the name of all of the "k" types under the "tag" tag of the nodes. These types will be audited to identify any dirty data.

In [29]:
def get_types(xml_file, tags):
    types = []
    node_types = set()
    way_types = {}
    for elem in get_element(xml_file, tags):
            for child in elem:
                if child.tag == "tag":
                    node_types.add(child.get('k'))
    return node_types

In [58]:
tags = ('node')
n = get_types(SAMPLE_OSM, tags)
pprint(n)

set(['addr:city',
     'addr:housenumber',
     'addr:postcode',
     'addr:province',
     'addr:street',
     'amenity',
     'barrier',
     'created_by',
     'crossing',
     'healthcare',
     'highway',
     'leisure',
     'name',
     'natural',
     'office',
     'place',
     'power',
     'railway',
     'ref',
     'shop',
     'source',
     'sport',
     'website'])


In [73]:
#finding problems in the addresses
#this is one example of finding all the variations in the street names
#similar scripts were executed to find the other problems outlined below
street = set() #for storing all of the different kinds of street names entered in the dataset
street2 = set()

for elem in get_element(SAMPLE_OSM, tags=('node', 'way')):
        for child in elem:
            if child.tag == "tag":
                if child.attrib['k'] == 'addr:street':
                    q = child.attrib['v'].split(' ')
                    if len(q) > 1:
                        street.add(q[1])
                    if len(q) > 2:
                        street2.add(q[2])
pprint(street)
print '\n'
pprint(street2)

set(['Avenue',
     'Common',
     'Crescent',
     'Dr.',
     'Drive',
     'Estates',
     'Glenmore',
     'Highway',
     'Hill',
     'Hills',
     'Manor',
     'Meadows',
     'Park',
     'Pine',
     'Road',
     'Street',
     'Trail',
     'Twintree',
     'Valley',
     'Way',
     'Wood'])


set(['',
     '51A',
     'Boulevard',
     'Circle',
     'Crescent',
     'Drive',
     'NE',
     'NW',
     'Northwest',
     'Park',
     'Place',
     'Road',
     'SE',
     'SW',
     'Southwest',
     'Way',
     'West'])


<p>A few of the above "k" attributes were explored for potential errors or inconsistencies in the data. The following section details the problems encountered and the solutions used.
</p>

## Problems Encountered in the Map

<ul>
    <li>Calgary with uppercase and lowercase 'c'</li>
    <li>Two different ways of writing the province ("AB" and "Alberta"). Contains an address from the province of Ontario that will need to be removed.</li>
    <li>Different ways of representing the quadrants in street names ("se", "SE", "S.E.", "SouthEast")</li>
    <li>Different ways of representing the same street names ("Dr.", "Drive"). Some street names have a semi-colon in the same entry.</li>
</ul>

These problems will be solved after the data in shaped into the desired schema in a python dictionary format

## Shape Element into Schema and Correct Problems

In [34]:
#set up the headings for node and way dictionaries and eventual csv transformation

NODE_FIELDS = ['id', 'user', 'uid', 'version', 'lat', 'lon', 'timestamp', 'changeset']
NODE_TAG_FIELDS = ['id', 'key', 'value', 'type']

WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']

key_re = re.compile(r'(.*?(?=:))\:(\S.+)') #regular expression for determining if colon exists in entry and to seperate 
#                                         # expression after first colon

In [35]:
#function to shape the node and way tags

def shape_tag(element, default_type = 'regular', key_re = key_re):
    tags = []
    
    for child in element:
        if child.tag == 'tag':
            tag= {}
            tag['id'] = element.attrib['id']
            tag['key'] = child.attrib['k']
            tag['value'] = child.attrib['v']
            tag['type'] = default_type
            
            key = key_re.search(child.attrib['k'])
            if key:
                tag['key'] = key.group(2)
                tag['type'] = key.group(1)
            
            tags.append(tag.copy())
    return tags
    

In [36]:
def shape_element(element, node_fields = NODE_FIELDS, way_fields = WAY_FIELDS, way_nodes = WAY_NODES_FIELDS, default_type = 'regular'):
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []
    
    if element.tag == "node":
        for field in node_fields:
            node_attribs[field] = element.attrib[field]
        
        tags = shape_tag(element)
                
        return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':
        for field in way_fields:
            way_attribs[field] = element.attrib[field]
        
        tags = shape_tag(element)
        
        i = 0
        for child in element:
            if child.tag == 'nd':
                i += 1
                way_node = {}
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = i
                way_nodes.append(way_node.copy())
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
    else:
        pass
                

In [71]:
#function for correcting values in a dictionary when a comparison map is presented

def correct_values(el, mapp):
    for tag in el['node_tags']:
        if tag['value'] in mapp:
            tag['value'] = mapp[tag['value']]
    return el

In [61]:
#set up lists to store the shaped node and way elements

nodes = []
ways = []

for elem in get_element(OSM_FILE):
    el = shape_element(elem)
    if elem.tag == 'node':
        nodes.append(el.copy())
    elif elem.tag == 'way':
        ways.append(el.copy())
    else:
        continue

In [86]:
#fixing the city names

mapping = {'calgary':'Calgary',
          'Calary': 'Calgary',
          'Calgary, AB': 'Calgary',
          'Foothills No. 31, M.D. of': 'Foothills County',
          'Rocky View No. 44, M.D. of': 'Rocky View County',
          'Rocky View': 'Rocky View County',
          'Rockyview County': 'Rocky View County'}

for el in nodes:
    el = correct_values(el, mapping)

In [66]:
#fixing province name

mapping = {'AB': 'Alberta',
          'Albertaa': 'Alberta',
          'ab': 'Alberta',
          'alberta': 'Alberta'}
#delete
de = ['ON']

for el in nodes:
    el = correct_values(el, mapping)
    for tag in el['node_tags']:
        if tag['value'] in de:
            nodes.remove(el)

In [75]:
# fixing all street and quadrant names

mapping = {'Northeast': 'NE',
          'NE;Trans-Canada': 'NE',
          'Northwest':'NW',
          'Southwest': 'SW',
          'South-west': 'SW',
          'Southeast': 'SE',
          'N.E.': 'NE',
          'n.e.\n': 'NE',
          'N.W.':'NW',
          'N.W': 'NW',
          'nw': 'NW',
          'se': 'SE',
          'S.E': 'SE',
          'S.W': 'SE',
          'South-east': 'SE',
          'St': 'Street',
          'st': 'Street',
          'Rd': 'Road',
          'rise': 'Rise',
          'N': 'North',
          'Blvd.': 'Boulevard',
          'Dr': 'Drive',
          'NW;Trans-Canada': 'NW',
          'Rroad': 'Road',
          'S': 'South',
          'Blvd.': 'Boulevard',
          'Dr.': 'Drive',
          'creek': 'Creek',
          'Tr': 'Trail'}

#find the third word in a street name
quad = re.compile(r'\b\S+.?$',re.IGNORECASE)

for node in nodes:
    for tag in node['node_tags']:
        if tag['key'] == ("street" or "street:name"):
            name = tag['value'].split(' ')
            for i, word in enumerate(name):
                if word in mapping:
                    name[i] = mapping[word]
                tag['value'] = ' '.join(name)

The code below checks if all of the undesired data values were cleaned or removed as intended

In [90]:
p_test = set()
c_test = set()
s_test = set()

def check_values(value, sett, elements = nodes):
    for elem in nodes:
        for tag in elem['node_tags']:
            if value in tag["key"]:
                w = tag['value'].split(' ')
                sett.add(w[-1])
    print "These are all of the {} in this data set: ".format(value)
    pprint(sett)
    print '\n'

check_values('province', p_test)
check_values('city', c_test)
check_values('street', s_test)

These are all of the province in this data set: 
set(['Alberta'])


These are all of the city in this data set: 
set(['',
     '#145',
     '15',
     '2',
     '20',
     '596',
     '6',
     '7',
     '8',
     'Airdrie',
     'Calgary',
     'Chestermere',
     'Cochrane',
     'County',
     'Creek',
     'I.D.',
     'Okotoks'])


These are all of the street in this data set: 
set(['',
     '204A',
     '205A',
     '21',
     '210',
     '210A',
     '210B',
     '224',
     '232',
     '234',
     '50A',
     '51',
     '51A',
     '52',
     '52A',
     '66',
     'AB',
     'Avenue',
     'Boulevard',
     'Close',
     'Common',
     'Court',
     'Crescent',
     'Drive',
     'East',
     'Gate',
     'Highway',
     'Hollow',
     'NE',
     'NW',
     'North',
     'Park',
     'Place',
     'Point',
     'Rise',
     'Road',
     'SE',
     'SW',
     'South',
     'Street',
     'Trail',
     'Way',
     'West',
     'high_mast'])




## Parsing information from XML file into CSV

Set up validator and unicode dictionary writer functions for transformation of clean data into csv file

In [51]:
SCHEMA = schema.schema

def validate_element(element, validator, schema=SCHEMA):
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement type {0} has errors: \n{1}"
        error_string = (
            "{0}: {1}".format(k,v if isinstance(v, str) else ', '.join(v)) for k,v in errors.iteritems()
        )
    
        raise cerberus.ValidationError(
            message_string.format(fields, '\n'.join(error_string))
    )

class UnicodeDictWriter(csv.DictWriter, object):
    
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
                k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k,v in row.iteritems()
            })
    
    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [89]:
#convert Python dictionaries into csv files after validating

with codecs.open(NODES_PATH, 'w') as nodes_file, \
    codecs.open(NODES_TAGS_PATH, 'w') as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'w') as ways_file, \
    codecs.open(WAY_NODES_PATH, 'w') as ways_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'w') as ways_tags_file:
        
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        nodes_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAG_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        ways_nodes_writer = UnicodeDictWriter(ways_nodes_file, WAY_NODES_FIELDS)
        ways_tags_writer = UnicodeDictWriter(ways_tags_file, WAY_TAGS_FIELDS)
        
        nodes_writer.writeheader()
        nodes_tags_writer.writeheader()
        ways_writer.writeheader()
        ways_nodes_writer.writeheader()
        ways_tags_writer.writeheader()
        
        validate = False
        validator = cerberus.Validator()
        
        for node in nodes:
            if validate:
                validate_element(node, validator)
            
            nodes_writer.writerow(node['node'])
            nodes_tags_writer.writerows(node['node_tags'])
        
        for way in ways:
            if validate:
                validate_element(way, validator)
            
            ways_writer.writerow(way['way'])
            ways_nodes_writer.writerows(way['way_nodes'])
            ways_tags_writer.writerows(way['way_tags'])

## Data Overview

In [2]:
#connect sqlite to python to run and modify queries in jupyter

import sqlite3 as sql
import pandas as pd

def return_query(query,alter=False): 
    cal_db = sql.connect("calgary_canada.db")
    c = cal_db.cursor()
    c.execute(query)
    if alter:
        c.commit()
    results = c.fetchall()
    r = pd.DataFrame(results)
    cal_db.close()
    return r

### Size of files

calgary_canada.db ............... 109 MB<br>
calgary_canada.osm .............. 165 MB<br>
nodes.csv ........................... 65 MB<br>
nodes_tags.csv .................. 2.4 MB<br>
ways.csv ........................ 8 MB<br>
ways_nodes.csv .................. 18 MB<br>
ways_tags.csv ................... 14 MB<br>

### Number of nodes

In [39]:
QUERY = "SELECT count(*) FROM nodes;"

<p class="num">755292</p>

### Number of ways

In [30]:
QUERY = "SELECT count(*) FROM way;"

<p class='num'>91236</p>

### Number of unique users

In [33]:
QUERY1 = "SELECT COUNT(DISTINCT(uid)) FROM nodes"

QUERY2 = "SELECT COUNT(DISTINCT(uid)) FROM way"

#### Nodes:
<p class='num'> 778</p>

#### Ways:
<p class='num'>619</p>

### Top 5 node contributing users

In [56]:
QUERY = '''
SELECT user, count(user) AS Total
FROM Nodes
GROUP BY user
ORDER BY Total DESC
LIMIT 5;
'''

a = return_query(QUERY)

print a

            0       1
0      sbrown  198212
1   Zippanova   68927
2      abDoug   45413
3  markbegbie   41948
4     dbo-osm   34347


### Select the node(s) with the most tags

In [49]:
QUERY = '''
SELECT nodes.id, count(node_tags.id) AS Count
FROM nodes JOIN node_tags
ON nodes.id = node_tags.id
GROUP BY nodes.id
ORDER BY count DESC
LIMIT 5;
'''
print return_query(QUERY)

          0   1
0  51970215  24
1  51971748  24
2  51972854  24
3  51969828  23
4  51970028  23


### Population centre with the most unique nodes

In [67]:
QUERY = '''
SELECT node_tags.value, count(*) as num
FROM node_tags
WHERE key='city'
GROUP BY value
ORDER BY num DESC
LIMIT 5;
'''

print return_query(QUERY)

                   0    1
0            Calgary  518
1  Rocky View County  243
2        Bragg Creek  179
3    Kananaskis I.D.   81
4    Tsuu T'ina #145   76


The City of Calgary has the most unique nodes, as expected, although the difference in the number of unique nodes is not as large as it should be. This indicates that "tag" tags have not been added to many nodes yet or the addresses of those nodes do not include the city name yet.

### Quadrant of the City of Calgary with the most addresses in database

In [15]:
QUERY_NE = '''
SELECT count(*) AS NE
FROM (SELECT value, count(value) as sum
FROM node_tags
WHERE key = 'street' AND value LIKE '%NE'
GROUP BY value);
'''

QUERY_NW = '''
SELECT count(*) AS NW
FROM (SELECT value, count(value) as sum
FROM node_tags
WHERE key = 'street' AND value LIKE '%NW'
GROUP BY value);
'''

QUERY_SE = '''
SELECT count(*) AS SE
FROM (SELECT value, count(value) as sum
FROM node_tags
WHERE key = 'street' AND value LIKE '%SW'
GROUP BY value);
'''

QUERY_SW = '''
SELECT count(*) AS SW
FROM (SELECT value, count(value) as sum
FROM node_tags
WHERE key = 'street' AND value LIKE '%SE'
GROUP BY value);
'''

no_display='''
print return_query(QUERY_NE)
print return_query(QUERY_NW)
print return_query(QUERY_SE)
print return_query(QUERY_SW)'''

#### NE:
<p class = 'num'>38</p>

#### NW:
<p class = 'num'>58</p>

#### SE:
<p class = 'num'>66</p>

#### SW:
<p class = 'num'>82</p>

### Top 10 Amenities

In [21]:
QUERY = '''
SELECT value, count(value) as num
FROM node_tags
WHERE key = 'amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;
'''

print return_query(QUERY)

              0    1
0    restaurant  422
1     fast_food  398
2          fuel  190
3         bench  182
4          cafe  175
5          bank  136
6      post_box   96
7       parking   85
8  waste_basket   76
9       toilets   73


It looks like there are more restaurants and fast food places in the city than every other amenity combined. Let's find the most popular types of cuisines.

### Most popular restaurant type

In [40]:
QUERY = '''
SELECT node_tags.value, COUNT(*) as num
FROM node_tags
    JOIN (SELECT id FROM node_tags WHERE value='restaurant') as i
    ON node_tags.id = i.id
WHERE node_tags.key='cuisine'
GROUP BY node_tags.value
ORDER BY num DESC
LIMIT 5;
'''

print return_query(QUERY)

            0   1
0     chinese  37
1  vietnamese  22
2    japanese  15
3       pizza  15
4     italian  14


## Additional Ideas

I think the biggest issue with cleaning the dataset of Calgary and surrounding area is a quadrant identifier in addresses that is only required for buildings inside the city boundary. One idea to overcome this issue is to create an additional "tag" tag that identifies the quadrant under the node of all addresses inside the City of Calgary's border. There should also be a guideline for entering the quadrant and other frequently used words to ensure consistency and reduce the amount of data cleaning required. The guideline should be a list with accepted values for street names and quadrants. An example guideline would be: 

########################################
#######&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Guideline&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;#######
########################################<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<u>Accepted Names</u><br>
"Boulevard"<br>
"Court"<br>
"Drive"<br>
"NE"<br>
...

Althought not all users would refer to the guideline when entering data, it would create a standard for entering commonly used words and should help in the data cleaning process by reducing the variations in the words.

### Considerations for adding a quadrant tag

The City of Calgary is unique in its area because it is the only municipality that uses a quadrant for every address inside the city. To improve the data in the local dataset and reduce data cleaning efforts, a quadrant identifier tag should be added for all nodes with addresses and the quadrant should be omitted from the address in the "addr:street" identifier.

#### Benefits
<ul>
<li>The "addr:street" identifier will have the same format ([street name] [street type]) for nodes inside and outside of the city</li>
<li>An "addr:quadrant" identifier can be queried if user wishes to obtain nodes from specific quadrants of the city.</li>
<li>Each quadrant can be seperately queried for data analysis purposes. For example, how many chinese restaurants are in the NE quadrant compared to the NW quadrant.</li>
</ul>

#### Anticipated Problems
<ul>
<li>There may be entries that do not include the quadrant in the "addr:street" identifier or the "addr:quadrant" identifier.</li>
<li>The incorrect quadrant might be entered into the "addr: quadrant" identifier.</li>
<li>Requires extra effort from the user because now they have to split up the address. Will be more difficult for users entering data with bots.</li>
</ul>

Overall, an extra quadrant tag may require a little more effort from the user and create some scenerios where addresses are entered incorrectly, I think that these problems can be solved with a little due diligence from the user. Having access to the quadrant apart from the street name will allow for much better querying of the data, reduce effort spent in data cleaning, and provide better insight of the city and surrounding areas.

### Other details noticed:
<ul>
    <li>The top 5 users contributed 51% of the data while the other 773 users contributed 49%</li>
    <li>Many of the "key" and "value" tags were entered in all lowercase letters and varying formats for similar entries </li>
    <li>The "city" key tag is randomly included for some addresses but not others. None of the top 5 contributors added addresses that included the "city" key</li>

</ul>

The above observations lead me to believe that most of the nodes that had user entered data in the "key" and "value" attributes were manually entered. To improve the quality of the data, there should be a reference guideline on entering street types and other commmon words, as mentioned above.

## Conclusion

The open street map data for the City of Calgary and surrounding areas contains a good overview of the city, although it is far from complete. The map contains the most addresses for the SW quadrant of the city (82) and the least for the NE (38). The data set also contains some addresses of surrounding counties and towns and a First Nation's reserve. Evidence in data entry suggests that most users entered information manually rather than deploying a bot because there are several variances of the same words. Data was suffienctly cleaned for the purposes of this exercise.

An interesting observation is that Calgary has a lot of restaurants and fast food places. One reason for this is the economic boom that the city had enjoyed for a long time as the oil and gas capital of Canada. The economic prosperity allowed many residents to have a large disposable income to spend on dining. It will be interesting to see how this number changes over the next several years now that Calgary has been in a recession sinces mid 2014. The recession is a result of the downturn in commidity prices and local businesses have closed as a result.

Overall I believe that the OSM data set for Calgary contains accurate information. It can be further improved by standardizing the way that manual user entries are added. Some of the dirty data that is currently in the dataset can also be cleaned using the cleaning functions included in this project.

## References

<ul>
<li><a href = "https://www.udacity.com/course/data-wrangling-with-mongodb--ud032">Udacity Data Wrangling Course</a></li>
<li><a href = "www.stackoverflow.com">Stack Overflow</a></li>
<li><a href = "https://regex101.com/#python">Regular Expressions</a></li>
<li><a href = "www.openstreetmap.org">Open Street Map</a></li>
<li><a href = "https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md">Udacity sample project</a></li>

</ul>